In [ ]:
# !pip install openai
# !pip install langchain
# !pip install langchain_openai

In [20]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] =userdata.get('Secret_key')
openai.api_key = os.getenv('OPENAI_API_KEY')

In [21]:
#모델 응답 >> 문자열 파싱(parsing) 하는데 사용
#언어모델(llm) 출력(응답) >> 텍스트 형식으로 변환

from langchain_core.output_parsers import StrOutputParser

#대화형 프롬프트 템플릿 정의
from langchain_core.prompts import ChatPromptTemplate

#openai의 언어 모델 사용 >> 대화형 응답 생성
from langchain_openai import ChatOpenAI

#객체 생성
chat_prompt_template = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser()
#message의 contents 추출 >> string(문자) 변환

In [22]:
#chain 정의 (아직 호출 전)
chain = chat_prompt_template | chat_model | output_parser

In [23]:
chain.invoke({'topic' : 'ice cream'})
#invoke의 역할 >> 인자를 dict로 넘겨줌 (dict {k:v} >> {'사과':'과일'})
#이유 ? prompt_template가 dict로 받기 때문

'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [24]:
chain.invoke({'topic' : 'ice cream'})


'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [25]:
print(chain.invoke('coding test'))


Why did the programmer break up with the coding test? Because it had too many bugs to work out!


In [26]:
#dict말고 icecream만 넣고 싶을때

from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

# runnables : 테스트 및 디버깅 용도
# RunnablePassthrough : 입력값을 그대로 출력값에 전달
# RunnableParallel : 여러 개 runnable 객체를 병렬 실행
# RunnableLambda : 람다 함수(임의 함수) 감싸서 사용

In [27]:
chat_prompt_template = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

In [28]:
chain = RunnablePassthrough() | chat_prompt_template | chat_model | output_parser

In [29]:
print(chain.invoke('ice cream'))

Why did the ice cream truck break down?

Because it had too many "scoops"!


In [30]:
chain.invoke({'topic' : 'ice cream'})

'Why did the ice cream truck break down? \n\nIt had too many sundaes!'

In [38]:
class Runnable:

    def __init__(self, func):
        self.func = func

    # 이 메서드는 파이썬의 비트 or 연산자(|)를 오버로드한다.
    def __or__(self, other):

        # 다른 함수가 이 함수의 결과를 활용
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))

        #러너블 객체로 감싸게 됨
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [39]:
#예시

def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

#Runnable로 함수 감싸기
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

In [40]:
chain = add_five | multiply_by_two
#(3 + 5) * 2
chain(3)

16

In [41]:
add_five.__or__(multiply_by_two)
chain(3)

16

In [42]:
def num2ko(x):
    return str(x) + '입니다.'

In [43]:
chain = add_five | multiply_by_two | num2ko

In [44]:
chain(3)

'16입니다.'

In [45]:
# 템플릿 만드는 방법

# 미션 : 어떤 주제에 대해서 사용자 레벨에 맞게 선택된 언어로 대화 예시를 만드는 앱을 개발!
# 미션 : [어떤] 주제에 대해서 사용자 [레벨]에 맞게 [선택된 언어]로 대화 예시를 만드는 앱을 개발!

# 기획
# "Please generate dialogue sentences in English on the topic of health for a beginner level."
# "Please generate dialogue sentences in Korean on the topic of AI for an intermediate level."

# 양식과 변수를 분리하기
# "Please generate three sentences in a dialogue in (English) on the topic of (health) for a (beginner) level."
# "Please generate three sentences in a dialogue in (Korean) on the topic of (AI) for an (intermediate) level."

# "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a/an {level} level."

# 양식 조정하기
# "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

In [47]:
from langchain_core.runnables import RunnablePassthrough

template = \
"Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."


In [48]:
# cf) chatgpt
# [초보자 수준]으로 [저녁메뉴] 관련된 3개의 문장으로 구성된 대화지문 [한국어]로 생성해줘.

In [49]:
chat_prompt_template = ChatPromptTemplate.from_template(template)

In [50]:
chain = chat_prompt_template | chat_model | StrOutputParser()

In [53]:
#변수는 dict 넣어야 함
output = chain.invoke({'language' : 'English', 'topic' : 'dinner menu', 'level' : 'beginner'})
print(output)

Person 1: What's for dinner tonight?
Person 2: We're having spaghetti and meatballs.
Person 1: Yum, that sounds delicious! What else is on the menu?


In [54]:
# 변수 중에 시스템에 입력해야 할 것과 사용자가 입력해야 할 것 분리
# language : 설정
# topic : 바뀐다
# level : 설정

def get_learning_language(_):
    print("###")
    print("_")
    print("in get_learning_language")
    print("###")
    return 'English'
# '_' 함수 내에서 사용되지 않는 값을 받을 때
# >> 여기서는 입력값을 사용하지 않고 항상 "English" 반환

def get_learning_level(_):
    print("###")
    print('_')
    print('in get_learning_level')
    print("###")
    return "advanced"

첫 번째 방법:

- 모든 데이터를 미리 구성해야 하므로, 간단한 전처리나 고정된 값을 사용할 때 직관적입니다.
- 데이터 전처리를 명시적으로 관리할 수 있어, 각 값이 어떻게 계산되는지 명확하게 보여줍니다.
- 모든 전처리를 호출 전에 완료해야 하므로, 복잡한 전처리 로직이 있을 때 코드가 길어질 수 있습니다



두 번째 방법:

- 전처리 단계에서 필요한 함수를 자동으로 호출할 수 있어 코드가 더 간결해질 수 있습니다.
- 입력값이 여러 단계의 변환이나 추가 작업이 필요할 때 유용합니다.
- 데이터가 체인 내부에서 동적으로 처리되므로, 동일한 체인을 다양한 입력값으로 재사용할 때 편리합니다.

In [55]:
# 호출 방법 1

template =\
"Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

chat_prompt_template = ChatPromptTemplate.from_template(template)

chain= chat_prompt_template | chat_model | StrOutputParser()

# invoke 하기 전에 필요한 정보를 dict 구성
output =chain.invoke({'language': get_learning_language(''), 'topic': 'travel', 'level': get_learning_level('')})

print(output)

###
_
in get_learning_language
###
###
_
in get_learning_level
###
Person 1: Have you ever been to Japan?
Person 2: Yes, I visited Tokyo last year and it was an amazing experience.
Person 1: That's great! I've always wanted to travel there. What was your favorite part of the trip?


In [56]:
# 호출방법 2 Runnable 사용

from langchain_core.runnables import RunnablePassthrough

template =\
"Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

chat_prompt_template = ChatPromptTemplate.from_template(template)

# 함수의 인자는 '무조건 1개' 이어야 함

chain= (
    RunnablePassthrough.assign(language=get_learning_language,
                               level=get_learning_level)
    |chat_prompt_template
    | chat_model
    | StrOutputParser()

)

# {'topic' : 'travel'} >> dict / language, level 추가 >> {"topic"....'language', ... 'level'} dict 완성

output = chain.invoke({'topic':'travel'}) # 변수 1개

print(output)

###
_
in get_learning_language
###
###
_
in get_learning_level
###
Person A: "I've been thinking about taking a trip to Japan next year. Have you ever been?"

Person B: "Yes, I actually lived in Japan for a year. It was such an amazing experience. You should definitely go!"

Person A: "That's incredible! I've always wanted to visit Tokyo and try authentic Japanese cuisine. Any recommendations?"


In [57]:
chain

RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
})
| ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}.'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ad27e8c6da0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ad27e0774f0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [58]:
# 호출방법 2 Runnable 사용

from langchain_core.runnables import RunnablePassthrough

template =\
"Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

chat_prompt_template = ChatPromptTemplate.from_template(template)

# 함수의 인자는 '무조건 1개' 이어야 함

chain= (
    RunnablePassthrough.assign(language=get_learning_language,
                               level=get_learning_level)
    |chat_prompt_template
    | chat_model
    | StrOutputParser()

)

# {'topic' : 'travel'} >> dict / language, level 추가 >> {"topic"....'language', ... 'level'} dict 완성

output = chain.invoke({'topic':'weather'}) # 변수 1개

print(output)

######
_
in get_learning_language
###

_
in get_learning_level
###
Person 1: "I can't believe how hot it is today. I feel like I'm melting."
Person 2: "I know, right? It's so humid too. I can't wait for a cool breeze to come through."
Person 1: "I heard there's a chance of thunderstorms later. I hope it cools things down a bit."
